# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content

In [27]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
import os

# Load a single PDF file
pdf_path = "../Introduction_to_Philosophy-WEB_cszrKYp-compressed.pdf"
pdf_loader = PyPDFLoader(pdf_path)
raw_docs = pdf_loader.load()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(raw_docs)

print(docs)

[Document(page_content='Introduction to Philosophy  \n \n \n \n \n \n \n \n \n \nSENIOR CONTRIBUTING AUTHOR  \nNATHAN SMITH , HOUSTON COMMUNITY COLLEGE', metadata={'source': '../Introduction_to_Philosophy-WEB_cszrKYp-compressed.pdf', 'page': 2}), Document(page_content='OpenStax  \nRice University  \n6100 Main Street MS -375 \nHouston, Texas 77005  \n \nTo learn more about OpenStax, visit http s://openstax.org.  \nIndividual print copies and bulk orders can be purchased through our website.  \n \n©2022  Rice University.  Textbook content produced by OpenStax is licensed under a Creative Commons \nAttribution 4.0 International License  (CC BY 4.0) . Under this license, any user of this textbook or the textbook \ncontents herein must provide proper attribution as follows:   \n \n- If you redistribute this textbook in a digital format (i ncluding but not limited to  PDF and HTML), then you \nmust retain on every page the follo wing attribution:  \n“Access for free at openstax.org .” \n- If

## Load the Content in a Vector Store

In [28]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.vectorstores import FAISS
# Define LLM
llm = OpenAIEmbeddings()

# Create a vector store from documents and the LLM
vector_store = FAISS.from_documents(docs, llm)

## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [29]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in docs], columns=["text"])
df.head(10)

,text
0,Introduction to Philosophy \n \n \n \n \n \n ...
1,OpenStax \nRice University \n6100 Main Stree...
2,following attribution: \n“Access for free at ...
3,Rice University logo are not subject to the l...
4,"OPENSTAX \n \nOpenStax provides free, peer -re..."
5,mission by cultivating a diverse community of ...
6,Charles Koch Foundation \nLeon Lowenstein Fou...
7,"Study where you want, what \nyou want, when yo..."
8,CONTENT S\nPrefac e 1\nCHAPTER 1\nIntroduction...
9,CHAPTER 3\nThe Earl y His tory of Philosoph y ...


We can now create a Knowledge Base using the DataFrame we created before.

In [30]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

## Generate the Test Set

In [31]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot designed to answer questions about philosophy material",
)

2024-06-10 23:12:03,832 pid:82744 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-06-10 23:13:59,246 pid:82744 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

Let's display a few samples from the test set.

In [32]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: What is the title of the book by Marcus Aurelius mentioned in the context?
Reference answer: The title of the book by Marcus Aurelius mentioned in the context is 'Meditations: The Annotated Edition'.
Reference context:
Document 651: 11.How did Ibn Sina ’s scientific appro ach diff er from tha t of the Aris totle and the Epicure ans?
Further R eading
Aurelius , Marcus . 2021. Medita tions: The Annota ted Edition . Transla ted and e dite d by Robin W aterfield . New
York: Basic Bo oks.
Berk ovits, Elie zer. 1961. “ Wha t Is J ewish Philosoph y?”Tradition: A J ournal o f Or thodox Jewish Thought 3 (2):
117–130. ht tps:/ /traditiononline .org/wha t-is-jewish-philosoph y/.
Golds tone , Jack A . 2009. Why Europ e? The Rise o f the W est in W orld His tory, 1500–1850 . Bos ton: McGra w-Hill
Higher E ducation .
Plato. (1888) 2017. The R epublic . Transla ted by Benjamin J owett. 3rd e d. Oxf ord: Clarendon P ress; Project
Gutenb erg. https:/ /www.gutenb erg.org/ebooks/55201.4 • F u

Let's now save the test set to a file:

In [33]:
testset.save("test-set.jsonl")

## Prepare the Prompt Template

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [39]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What is the Machine Learning School?")

[Document(page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve real-world problems is hard.I want to change that.I started writing software 30 years ago. I\'ve written pipelines and trained models for some of the largest companies in the world. I want to show you how to do the same.This is the class I wish I had taken when I started.This program will help you unlearn what you think machine learning is. It\'s a practical, hands-on class where you\'ll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions.

We can now create our chain.

In [45]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.

In [37]:

import requests
import uuid

def invoke(data):
    # Generate a random prepareKey
    question = data["question"]
    prepare_key = str(uuid.uuid4())

    # Define the API endpoint and the JSON body
    api_url = "http://localhost:8001/api/v1/openai/ask"
    json_body = {
        "data": {
            "content": f"""you have to answer the question base on the uploaded file, and the file have already uploaded {question}""",
            "prepareKey": prepare_key,
            "files": []
        },
        "maxToken": 2000,
        "currentUser": {
            "globalName": "Cảnh",
            "username": ".canh"
        },
        "type": "discord"
    }

    # Make the POST request
    response = requests.post(api_url, json=json_body)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        print(data["data"])
        return data["data"]
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

invoke({"question": "What is the file mainly about"})


The file is mainly about how to approach and navigate a philosophy text, as well as providing an introduction to philosophy and its various topics. It also includes information on reading philosophy effectively and engaging with ideas and arguments.


'The file is mainly about how to approach and navigate a philosophy text, as well as providing an introduction to philosophy and its various topics. It also includes information on reading philosophy effectively and engaging with ideas and arguments.'

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [38]:
def answer_fn(question, history=None):
    return invoke({"question": question})
    

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [39]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/60 [00:00<?, ?it/s]

The title of the book by Marcus Aurelius mentioned in the context is "Meditations."
Foot argues that understanding what is good for an organism involves knowing what is good for it based on its vital processes and nature. By studying an organism's nature, one can determine what is beneficial for it and contributes to its well-being.
In the first chapter of Laozi's Daodejing, the Tao is described as enduring and unchanging. It is said that the Tao cannot be trodden or named. It is the originator of heaven and earth and the mother of all things.
After the Spanish conquest of the Maya territory, Catholic priests burned almost all of the Maya codices as well as their scientific and technical manuals.
According to the context, the 'Inclusive Care' and 'Condemning Aggression' doctrines are significant because they emphasize the importance of considering and caring for others. The Mohist philosophy believes that every human being is valued equally in the eyes of heaven, and that partiality in

CorrectnessMetric evaluation:   0%|          | 0/60 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [40]:
display(report)

Loading BokehJS ...

In [41]:
report.to_html("report.html")


We can display the correctness results organized by question type.

In [42]:
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
conversational,0.1
distracting element,0.8
double,0.7
simple,0.9
situational,0.9


We can also display the specific failures.

In [43]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
d9b2d507-981e-4d77-82ef-1c6893dd21b1,What is the title of the book by Marcus Aureli...,The title of the book by Marcus Aurelius menti...,Document 651: 11.How did Ibn Sina ’s scientifi...,[],"{'question_type': 'simple', 'seed_document_id'...",The title of the book by Marcus Aurelius menti...,False,The agent did not provide the full title of th...
a89c9e05-cf40-441f-ab4c-8142c7154373,"In light of Plato's Allegory of the Cave, how ...",The main areas of bioethical debate include ab...,Document 1530: Summary\n10.1 The Challenge of ...,[],"{'question_type': 'distracting element', 'seed...","Based on the document, there is no mention of ...",False,The agent's answer does not address the questi...
3574dfa6-33ec-47f8-8086-ad4222b62b28,Considering the principles of gender binarism ...,The ethical obligations of software engineers ...,Document 1503: Society (IEEE -CS) pro vides a ...,[],"{'question_type': 'distracting element', 'seed...","Based on the provided information, there is no...",False,The agent's answer does not address the influe...
97aa3789-7629-4548-9773-528ef8e45785,"Hey, I'm a philosophy student and I have an ex...",The main areas of bioethical debate include ab...,Document 1530: Summary\n10.1 The Challenge of ...,[],"{'question_type': 'situational', 'seed_documen...",The main areas of bioethical debate according ...,False,The agent's answer is partially correct but it...
2fda232e-f37b-490b-a5e9-794446357bcb,What is the source of information about the ca...,The source of information about the caste syst...,"Document 1698: Johnson , Donald , and J ean J ...",[],"{'question_type': 'double', 'original_question...","Based on the document, the source of informati...",False,The agent did not provide the correct source o...
f428d1cc-0e24-4ec9-9785-23c362021339,Who edited the 'Encyclopedia of Philosophy' an...,The 'Encyclopedia of Philosophy' was edited by...,"Document 1564: Ency clop edia o f Philosoph y,...",[],"{'question_type': 'double', 'original_question...","Based on the document provided, the informatio...",False,The agent failed to provide the correct inform...
8be49adb-f29c-4917-acf2-a815fa4965dc,What is the main difference between act and ru...,This context does not provide an answer.,"Document 1394: 7.For utilitarians , which c on...",[],"{'question_type': 'double', 'original_question...",The main difference between act and rule utili...,False,The agent provided an answer while the ground ...
3f655eac-ec93-4ff1-9427-95978361f9dc,Can you provide its definition?,A global skeptic is someone who rejects the po...,Document 1070: beliefs c onfer jus tific ation...,"[{'role': 'user', 'content': 'I'm interested i...","{'question_type': 'conversational', 'seed_docu...","I'm sorry, but I cannot provide the definition...",False,The agent failed to provide the definition ask...
4776365c-6028-410a-9d4c-d19ef244a109,Could you tell me the main difference?,Internalism is the view that justification for...,Document 984: cannot nowrecall wha t tha t sou...,"[{'role': 'user', 'content': 'I'm interested i...","{'question_type': 'conversational', 'seed_docu...",The main difference between act and rule utili...,False,The agent's answer is incorrect because it exp...


## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [22]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

Create a Test Suite from the test set.

In [23]:
test_suite = testset.to_test_suite("Machine Learning School Test Suite")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [24]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [25]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Machine Learning School Question and Answer Model",
    description="This model answers questions about the Machine Learning School website.",
    feature_names=["question"], 
)

2024-03-23 16:20:54,903 pid:46357 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [64]:
test_suite_results = test_suite.run(model=giskard_model)

2024-03-23 15:57:39,422 pid:2158 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-03-23 15:57:39,423 pid:2158 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:00.007341
Executed 'TestsetCorrectnessTest' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x2fa779b50>, 'dataset': <giskard.datasets.base.Dataset object at 0x2fa899b80>}: 
               Test succeeded
               Metric: 0.62
               
               
2024-03-23 15:58:58,289 pid:2158 MainThread giskard.core.suite INFO     Executed test suite 'Machine Learning School Test Suite'
2024-03-23 15:58:58,291 pid:2158 MainThread giskard.core.suite INFO     result: success
2024-03-23 15:58:58,292 pid:2158 MainThread giskard.core.suite INFO     TestsetCorrectnessTest ({'model': <giskard.models.function.PredictionFunctionModel object at 0x2fa779b50>, 'dataset': <gi

We can display the results.

In [65]:
display(test_suite_results)

<TestSuiteResult (passed)>

## Integrating with Pytest

In [27]:
import ipytest

We can now integrate our test suite with Pytest.

In [36]:
%%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()

.2024-03-23 16:27:56,471 pid:46357 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-03-23 16:27:56,472 pid:46357 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:00.005269


.                                                                                           [100%]
========================================= warnings summary =========================================
../.venv/lib/python3.9/site-packages/_pytest/config/__init__.py:1276
  /Users/svpino/dev/llm/.venv/lib/python3.9/site-packages/_pytest/config/__init__.py:1276: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: anyio
    self._mark_plugins_for_rewrite(hook)

t_66406511b9d84eb38baa6b0a22141dd0.py::test_llm_correctness
  /Users/svpino/dev/llm/.venv/lib/python3.9/site-packages/_pytest/python.py:199: PytestReturnNotNoneWarning: Expected None, but t_66406511b9d84eb38baa6b0a22141dd0.py::test_llm_correctness returned To execute the test call "execute()" method, which will be an error in a future version of pytest.  Did you mean to use `assert` instead of `return`?
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
2 passed, 2 warni